# Saving Arrays to HDF5

The Hierarchical Data Format was created by the National Center for Supercomputing Applications (NCSA) in 1987. It consists of *datasets*, which are multidimensional arrays of a single data type, and *groups*, which are container structures holding datasets and other groups.  Both groups and datasets may have attributes attached to them, which are any pieces of named metadata.  What this, in effect, does is to emulate a filesystem within a single file.  The nodes or "files" within this virtual filesystem are array objects.  Generally, a single HDF5 file will contain a variety of related data for working with the same underlying problem.

The Network Common Data Form (NetCDF) is a library of functions for storing and retrieving array data.  The project itself is nearly as old as HDF, and is an open standard that was developed and supported by a variety of scientific agencies. As of its version 4, it supports using HDF5 as its storage backend.

Generic HDF5 files typically have an extension of `.h5`, `.hdf5`, `.hdf`, or `.he5`.  These should all represent the same binary format, and other extensions occur sometimes too. Some corresponding extensions for HDF4 also exist.  Even though NetCDF can consist of numerous underlying file formats, they all seem standardized on the `.nc` or `.nc4` extension.

# Lazy Data Access

HDF5 is often used to store large datasets, up to gigabytes or terabytes in size.  Therefore, accessing a dataset is performed internally as a memory map to a relevant disk region rather than an eager read of the data into memory.  Operations like NumPy slices into an array only concretize into RAM the specific values needed for the action.

Within Python, there are two popular open source libraries for working with HDF5, **PyTables** and **h5py**. PyTables and h5py have moderately different attitudes.  H5py stays close to HDF5 spec itself while PyTables attempts to provide a higher-level "Pythonic" interface that was, like `lxml.objectify`, inspired by my defunct `gnosis.xml.objectify` library from 2000.  In this lesson, we use `h5py`.

For illustration, let us look at a 42 MiB NetCDF/HDF5 file published by NASA that contains satellite measurements.
The names for data files used by NASA are verbose but contain detailed indication in the names themselves of the nature of the data sets in them.  Displayed is the data set name, its dimensions, its data type, its shape, and the 'units' attribute that these all have.  In general, attributes may have any names, but NASA has conventions about which to use.

In [1]:
import numpy as np
import h5py
h5fname = ('/mnt/sda1/data/OMI-Aura_ANC-OMVFPITMET'
           '_2020m0216t225854-o82929_v003'
           '-2020m0217t090311.nc4')

data = h5py.File(h5fname, mode='r')

for name, arr in data.items():
    print(f"{name:6s} | {str(arr.shape):14s} | "
          f"{str(arr.dtype):7s} | {arr.attrs['units'][0]}")

DELP   | (1494, 60, 47) | float32 | Pa
PBLTOP | (1494, 60)     | float32 | Pa
PHIS   | (1494, 60)     | float32 | m+2 s-2
PS     | (1494, 60)     | float32 | Pa
T      | (1494, 60, 47) | float32 | K
TROPPB | (1494, 60)     | float32 | Pa
U      | (1494, 60, 47) | float32 | m s-1
U10M   | (1494, 60)     | float32 | m s-1
V      | (1494, 60, 47) | float32 | m s-1
V10M   | (1494, 60)     | float32 | m s-1
lat    | (1494, 60)     | float32 | degrees_north
lev    | (47,)          | int16   | 1
line   | (1494,)        | int16   | 1
lon    | (1494, 60)     | float32 | degrees_east
sample | (60,)          | int16   | 1
time   | (1494,)        | float64 | seconds since 1993-01-01 00:00:00


We can lazily create a memory view into only part of one of the dataset arrays.  In the example, we have opened in read-only mode, but if we had opened using the `'r+'` or `'a'` modes we could change the file.  Use the `'w'` mode with extreme caution since it will overwrite an existing file.  If the mode allowed modification on disk, calling `data.flush()` or `data.close()` will write any changes back to the HDF5 source.

Let us create a view of only a small section of the 3-dimensional V dataset.  We are not particularly concerned here with understanding the domain of the data, but just demonstrating the APIs.  In particular, notice that we have used a stride in one dimension to show that the general NumPy style of complex memory views is available.  Only the data referenced is actually put into main memory, the rest stays on disk.

In [2]:
# A 3-D block from middle of DELP array
middle = data['V'][::500, 10:12, :3]
middle

array([[[17.032158  , 12.763597  ,  3.7710803 ],
        [16.53227   , 12.759642  ,  4.1722884 ]],

       [[ 4.003829  , -1.0843939 , -6.7918572 ],
        [ 3.818467  , -1.0030019 , -6.6708655 ]],

       [[-2.7798688 ,  0.24923703, 20.513933  ],
        [-2.690715  ,  0.2226392 , 20.473366  ]]], dtype=float32)

If we modify the data in the view `middle`, it will be written back when we flush or close the handle (if not in read-only mode).  We might also use our data slice for other computations or data science purposes.  For example, perhaps such a selection acts as tensors input into a neural network.  In a simpler case, perhaps we simply want to find some statistics or reduction on the data.

In [3]:
middle.mean(axis=1)

array([[16.782215  , 12.76162   ,  3.9716845 ],
       [ 3.911148  , -1.0436978 , -6.7313614 ],
       [-2.735292  ,  0.23593812, 20.493649  ]], dtype=float32)

# Creating a New HDF5 File

Creating arrays within an HDF5 initially only allocates the disk sectors where they will live.  Filling them with data is a separate step.  This is similar to NumPy's `np.empty()` constructor that may allow random values to occur before an array is filled.

In [4]:
# Create a brand new file (possibly delete an existing one)
f = h5py.File('data/hierarchy.h5', 'w')

In [5]:
# A 4-D array of integers
f.create_dataset('/deeply/nested/group/my_data',
                 (10, 10, 10, 10), dtype='i')
# A 1-D array of integers
f.create_dataset('deeply/path/elsewhere/other', (20,), dtype='i')
# A 2-D array of floating point numbers
f.create_dataset('deeply/path/that_data', (5, 5), dtype='f');

In [6]:
# Assigning a group to a variable
dset = f['/deeply/nested/group/my_data']
# Fill the array with random values
dset[...] = np.random.randint(0, 99, (10, 10, 10, 10))
# Add some attributes to this dataset
dset.attrs['author'] = 'David Mertz'
dset.attrs['citation'] = 'INE Training'
dset.attrs['shape_type'] = '4-D integer array'
# Close file handle and write pending data
f.close()

## Reading Back Our New Serialization

Having serialized several datasets (arrays) to a new HDF5 file, we can open it in `'r+'` mode that will allow modification, but will not overwrite the entire file.

In [7]:
f = h5py.File('data/hierarchy.h5', 'r+')
# Assign a nested dataset to a variable
dset = f['/deeply/nested/group/my_data']
print(dset.shape, dset.dtype)

(10, 10, 10, 10) int32


In [8]:
for key, val in dset.attrs.items():
    print(key, "🠆", val)
print()
print("Data block:\n", dset[5:6, 3:4, 2:4, 8:])

author 🠆 David Mertz
citation 🠆 INE Training
shape_type 🠆 4-D integer array

Data block:
 [[[[22 76]
   [91 90]]]]


We can look at other groups and datasets as well.

In [9]:
# A different dataset in nested groups
for group in f['deeply/path']:
    print(group)
f['deeply/path/elsewhere/other']

elsewhere
that_data


<HDF5 dataset "other": shape (20,), type "<i4">

Or find all objects (groups and datasets) within the overall HDF5 file.

In [10]:
everything = []
f.visit(everything.append)
everything

['deeply',
 'deeply/nested',
 'deeply/nested/group',
 'deeply/nested/group/my_data',
 'deeply/path',
 'deeply/path/elsewhere',
 'deeply/path/elsewhere/other',
 'deeply/path/that_data']

## Modifying Existing Dataset

We see that we have a 4-dimensional array of integer data.  Perhaps some metadata description was attached to it as well.  Let us also view—and then modify—some section of the data.  After we change the data, we can write it back to disk.  We can similarly change or add attribues in a regular dictionary style.

In [11]:
dset.attrs['author'] = "David Q Mertz"
dset.attrs['negative_elements'] = True

Let us modify the same slice of data we displayed, then close the file handle to write it back to disk.

In [12]:
dset[5:6, 3:4, 2:4, 8:] = np.random.randint(-99, 0, (1, 1, 2, 2))
print("Data block:\n", dset[5:6, 3:4, 2:4, 8:])
# write change to disk
f.close()  

Data block:
 [[[[-69 -32]
   [-20 -60]]]]


Looking at our changes to verify:

In [13]:
f = h5py.File('data/hierarchy.h5', 'r+')
dset = f['/deeply/nested/group/my_data']
print("Superset data block:\n", dset[5:6, 3:4, 1:5, 7:])

Superset data block:
 [[[[ 60  67  97]
   [ 59 -69 -32]
   [ 84 -20 -60]
   [ 36  31  27]]]]


In [14]:
for key, val in dset.attrs.items():
    print(key, "🠆", val)

author 🠆 David Q Mertz
citation 🠆 INE Training
negative_elements 🠆 True
shape_type 🠆 4-D integer array
